# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [261]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="fltk-group-11"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Scale the default-node-pool up.

In [328]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 2 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 2 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/fltk-group-11/zones/us-central1-c/clusters/fltk-testbed-cluster].
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/fltk-group-11/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [329]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test experiment-extractor

Error: uninstall: Release not loaded: experiment-extractor: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [302]:
helm ls -n test

NAME      	NAMESPACE	REVISION	UPDATED                                 	STATUS  	CHART                       	APP VERSION
nfs-server	test     	1       	2022-09-22 17:21:31.971502719 +0200 CEST	deployed	nfs-server-provisioner-1.1.3	2.3.0      


In [330]:
helm upgrade --install -n test experiment-extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

Release "experiment-extractor" does not exist. Installing it now.
NAME: experiment-extractor
LAST DEPLOYED: Thu Oct  6 20:35:36 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


In [305]:
export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006

http://localhost:6006/
Forwarding from [::1]:6006 -> 6006
E1005 16:59:42.763253   70349 portforward.go:233] lost connection to pod


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [161]:
EXPERIMENT_FILE="../configs/distributed_tasks/example_arrival_config.json"
CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [312]:
helm uninstall -n test experiment-orchestrator

release "experiment-orchestrator" uninstalled


In [331]:
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

NAME: experiment-orchestrator
LAST DEPLOYED: Thu Oct  6 20:35:53 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [377]:
kubectl get pods -n test

NAME                                                     READY   STATUS      RESTARTS   AGE
fl-extractor-5d555cdb84-fjq9b                            1/1     Running     0          27m
fl-server                                                0/1     Completed   0          27m
nfs-server-nfs-server-provisioner-0                      1/1     Running     0          160m
trainjob-5942b8f9-c727-4b7c-a897-54eb15e9b56a-master-0   0/1     Completed   0          23m
trainjob-5942b8f9-c727-4b7c-a897-54eb15e9b56a-worker-0   0/1     Completed   0          23m
trainjob-79fd6aa7-8be2-4df1-8be6-78af665f4fc8-master-0   0/1     Completed   0          23m
trainjob-79fd6aa7-8be2-4df1-8be6-78af665f4fc8-worker-0   0/1     Completed   0          23m
trainjob-7cfef02f-fb1e-4283-8792-102764db7137-master-0   0/1     Completed   0          23m
trainjob-7cfef02f-fb1e-4283-8792-102764db7137-worker-0   0/1     Completed   0          23m
trainjob-8b94bb13-7af9-44c2-b11b-07b94486f507-master-0   0/1     Completed   0 

In [375]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

In [376]:
# To get logs from learners (example)
kubectl logs -n test trainjob-79fd6aa7-8be2-4df1-8be6-78af665f4fc8-master-0   
# To get logs from learners (federated learning)


10-06-2022 18:40:18 root         INFO     Loading file config/configuration.fltk.json
10-06-2022 18:40:18 root         INFO     Starting in client mode
10-06-2022 18:40:18 root         INFO     Starting with host=trainjob-79fd6aa7-8be2-4df1-8be6-78af665f4fc8-master-0 and port=23456
10-06-2022 18:40:18 root         INFO     Initializing backend for training process: gloo
10-06-2022 18:40:22 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 0
10-06-2022 18:40:22 torch.distributed.distributed_c10d INFO     Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 2 nodes.
10-06-2022 18:40:22 root         INFO     Starting Creating client with 0
10-06-2022 18:40:22 Client-0-79fd6aa7-8be2-4df1-8be6-78af665f4fc8 INFO     Initializing learning client
10-06-2022 18:40:23 root         INFO     Getting net: Nets.fashion_mnist_cnn
10-06-2022 18:40:23 Client-0-79fd6aa7-8be2-4df1-8be6-78af665f4fc8 INFO     Preparing learner model 

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [378]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ./logging_batch

tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [323]:
helm uninstall -n test experiment-orchestrator

release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [324]:
helm uninstall experiment-extractor -n test

release "experiment-extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [325]:
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

pytorchjob.kubeflow.org "trainjob-3d330c98-29f9-4cc7-a335-2aa669b8bf5b" deleted
pytorchjob.kubeflow.org "trainjob-63353c95-3ebd-406f-beeb-d743af5fc79b" deleted
pytorchjob.kubeflow.org "trainjob-6a851cd6-ff87-4aec-9496-41253d9900b8" deleted
pytorchjob.kubeflow.org "trainjob-6f739053-97e9-46b9-a139-3d01e79c09e5" deleted
pytorchjob.kubeflow.org "trainjob-7496757f-227e-43f6-b000-e7b597f624ea" deleted
pytorchjob.kubeflow.org "trainjob-9361188a-9990-416d-9e4b-862e5b5496f3" deleted


In [326]:
# This will remove all information and logs as well.

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/fltk-group-11/zones/us-central1-c/clusters/fltk-testbed-cluster].
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/fltk-group-11/zones/us-central1-c/clusters/fltk-testbed-cluster].
